In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats



from pandas.plotting import scatter_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.datasets import make_classification
from sklearn.metrics import precision_recall_curve, roc_curve, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, auc

from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import sys
sys.path.insert(0, '../src')
sys.path.insert(0, '../data')

import functions


%load_ext autoreload
%autoreload 2

## Data Import

Psychological Operations Assessment and Selection (POAS)

In [2]:
filepath = '../data/poas'
filenames = ['poas16.xlsx', 'poas17.xlsx', 'poas18.xlsx', 'poas19.xlsx', 'poas20.xlsx', 'poas21.xlsx', 'poas22.xlsx']
poas = functions.read_excel_files(filepath, filenames)
poas.sample(10)

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,RANK,RACE,MOS,SEC,AB,RGR,PT,ED,MARITAL,DEP,...,LANG,GT,EL,SC,CO,FA,ST,WVR,RESULT,CODE
1207,PFC,W,18X,T,T,F,248.0,HSD,S,0.0,...,NaN,111,109.0,109.0,109.0,109.0,110.0,T,INVOLUNTARY WITHDRAWL,IVW
1691,SPC,A,92A,T,F,F,244.0,6YR,S,0.0,...,CHINESE-MANDARIN (CHINESE-KUO-YU AND HSIANG),140,134.0,136.0,136.0,138.0,139.0,T,INVOLUNTARY WITHDRAWL,IVW
1256,SPC,I,88M,T,F,F,267.0,2YR,M,4.0,...,NaN,117,122.0,119.0,120.0,119.0,119.0,T,SELECT,SEL
836,SPC,B,92G,T,F,F,254.0,HSD,S,0.0,...,,115,112.0,113.0,114.0,114.0,115.0,F,VW,VW
1459,SPC,W,11B,F,T,F,292.0,4YR,M,0.0,...,NaN,130,124.0,124.0,122.0,123.0,125.0,T,MEDICAL,MED
1595,SPC,W,13M,T,F,F,249.0,HSD,M,1.0,...,NaN,129,124.0,123.0,124.0,124.0,126.0,F,SELECT,SEL
2282,SGT,W,11B,T,F,F,272.0,HSD,M,0.0,...,NaN,105,102.0,102.0,102.0,103.0,104.0,F,MEDICAL DROP,MED
2413,PFC,W,18X,T,T,F,288.0,1 YR,S,1.0,...,NaN,132,129.0,127.0,127.0,128.0,129.0,T,NON-SELECT,BRD
689,SPC,W,18X,T,T,F,247.0,HSD,S,0.0,...,,132,142.0,141.0,143.0,143.0,140.0,F,APFT FAILURE,APFT
2024,SPC,W,74D,T,F,F,258.0,4YR,M,2.0,...,SPANISH,101,98.0,99.0,97.0,98.0,100.0,F,PFA FAILURE,PFA


Special Forces Assesment and Selection (SFAS)

In [3]:
filepath = '../data/sfas'
filenames = ['sfas16.xlsx', 'sfas17.xlsx', 'sfas18.xlsx', 'sfas19.xlsx', 'sfas20.xlsx', 'sfas21.xlsx', 'sfas22.xlsx']
sfas = functions.read_excel_files(filepath, filenames)
sfas.sample(10)

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


,RANK,RACE,MOS,SEC,AB,RGR,PT,ED,MARITAL,DEP,...,LANG,GT,EL,SC,CO,FA,ST,WVR,RESULT,CODE
1073,SSG,W,11B,T,F,F,260,HSD,M,0.0,...,NaN,110.0,104.0,102.0,104.0,104.0,103.0,F,SELECTED,SEL
6571,SSG,B,11B,T,T,F,270.0,2 YR,D,1.0,...,NaN,117.0,109.0,110.0,107.0,107.0,111.0,F,+COVID-19,CV19
2683,SPC,W,35F,T,F,F,279,HSD,M,1.0,...,,114.0,108.0,110.0,108.0,109.0,111.0,F,SELECTED,SEL
5684,SPC,W,19D,T,T,F,271.0,1YR,S,0.0,...,NaN,121.0,117.0,117.0,117.0,117.0,119.0,F,MEDICAL DROP,MED
4443,SPC,W,13F,T,T,F,300.0,4YR,S,0.0,...,NaN,112.0,104.0,106.0,103.0,104.0,106.0,T,MEDICAL DROP,MED
2721,SGT,W,11B,T,F,F,254,HSD,S,1.0,...,,111.0,109.0,110.0,109.0,109.0,110.0,F,INVOLUNTARY WITHDRAW,IVW
7053,SSG,A,11B,T,T,F,290.0,1 YR,S,0.0,...,NaN,105.0,109.0,110.0,110.0,109.0,108.0,F,PFA FAILURE,PFA
3934,SGT,W,35P,T,T,T,300.0,HSD,M,1.0,...,ARABIC-MODERN STANDARD,133.0,122.0,123.0,122.0,122.0,126.0,F,SELECTED,SEL
7273,SPC,W,42A,T,F,F,250.0,HSD,S,0.0,...,SPANISH,111.0,84.0,86.0,81.0,82.0,90.0,F,PFA FAILURE,PFA
4191,SPC,B,94F,T,F,F,248.0,HSD,S,0.0,...,NaN,116.0,106.0,107.0,106.0,107.0,110.0,F,APFT FAILURE,APFT
